In [1]:
""" water movement within the root (static soil) """


import sys;
import os
currentpath = os.getcwd()
if currentpath[-4:] == "tBox":
    path_ = "/applications/phloem_flow/"
    CPBdir = ""
else:
    path_ = ""
    CPBdir = "../.."

#directoryN = "/"+sys.argv[0].split('.')[0]+"/"
directoryN = "/branching_noCnoW/"
sys.path.append(path_+"../.."); sys.path.append(path_+"../../src/python_modules")
sys.path.append('/home/rbtlm640/4UQ/CPlantBox/src/python_modules')
#import matplotlib
#matplotlib.use('AGG') #otherwise "import matplotlib.pyplot" hangs

from phloem_flux import PhloemFluxPython  # Python hybrid solver
#from Leuning_speedup import Leuning #about 0.7 for both
#from photosynthesis_cython import Leuning
import plantbox as pb
#from plantbox import Photosynthesis as ph
#import vtk_plot as vp
import math
import os
import numpy as np
import vtk_plot as vp

#import matplotlib.pyplot as plt
from datetime import datetime, timedelta


def delete_verbose():
    home_dir = os.getcwd()
    dir_name = ""#"/results"
    dir_name2 = home_dir + dir_name
    test = os.listdir(dir_name2)
    for item in test:
        if item.startswith("outphoto"):
            os.remove(os.path.join(dir_name2, item))
        if item.startswith("fluxesphoto"):
            os.remove(os.path.join(dir_name2, item))
        if item.startswith("loopphoto"):
            os.remove(os.path.join(dir_name2, item))
        if item.startswith("errphoto"):
            os.remove(os.path.join(dir_name2, item))
        if item.startswith("/results"+directoryN+"prints"):
            os.remove(os.path.join(dir_name2, item))
#delete_verbose()


isCluster = (os.environ['HOME'] == '/home/m.giraud')
if False:
    def print(*args, **kwargs):
        """ custom print() function.
            for cluster: can get output even if program stop
            unexpectedly (i.e., without creating the outputfile)
        """
        # Adding new arguments to the print function signature
        # is probably a bad idea.
        # Instead consider testing if custom argument keywords
        # are present in kwargs
        if 'sep' in kwargs:
            sep = kwargs['sep']
        else:
            sep = ' '
        home_dir = os.getcwd()
        dir_name =  "/results"+directoryN
        dir_name2 = home_dir + dir_name
        name2 = dir_name2 + 'prints.txt'
        with open(name2, 'a') as log:
            for arg in args: log.write(str(arg) + sep)
            log.write('\n')

#time lapse pea plants:
#https://www.youtube.com/watch?v=a2Mxpg5lNFM
#       qr, qs, alpha, n, ks (in [cm/d])
#vgSoil = [0.059, 0.45, 0.00644, 1.503, 1]
def theta2H(vg,theta):#(-) to cm
    thetar =vg[0]# 0.059
    thetas = vg[1]#0.445
    alpha = vg[2]#0.00644
    n = vg[3]#1.503
    nrev = 1/(1-1/n)
    H =-(((( (thetas - thetar)/(theta - thetar))**nrev) - 1)**(1/n))/alpha
    return(H)#cm

def sinusoidal(t):
    return (np.sin(np.pi*t*2)+1)/2

def qair2rh(qair, es_, press):
    e =qair * press / (0.378 * qair + 0.622)
    rh = e / es_
    rh=max(min(rh, 1.),0.)
    return rh


def weather(simDuration):
    vgSoil = [0.059, 0.45, 0.00644, 1.503, 1]
    
    Qmin = 0; Qmax = 960e-6 #458*2.1
    Tmin = 15.8; Tmax = 22
    specificHumidity = 0.0097
    Pair = 1010.00 #hPa
    thetaInit = 30/100

    coefhours = sinusoidal(simDuration)
    TairC_ = Tmin + (Tmax - Tmin) * coefhours
    Q_ = Qmin + (Qmax - Qmin) * coefhours
    cs = 850e-6 #co2 paartial pressure at leaf surface (mol mol-1)
    #RH = 0.5 # relative humidity
    es =  6.112 * np.exp((17.67 * TairC_)/(TairC_ + 243.5))
    RH = qair2rh(specificHumidity, es, Pair)
    
    pmean = theta2H(vgSoil, thetaInit)
    
    weatherVar = {'TairC' : TairC_,
                    'Qlight': Q_,
                    'cs':cs, 'RH':RH, 'p_mean':pmean, 'vg':vgSoil}
    print("Env variables at", round(simDuration//1),"d",round((simDuration%1)*24),"hrs :\n", weatherVar)
    return weatherVar

def div0(a, b, c):        
    return np.divide(a, b, out=np.full(len(a), c), where=b!=0)
    
def div0f(a, b, c):    
    if b != 0:
        return a/b
    else:
        return a/c
        
def write_file_array(name, data):
    name2 = 'results'+ directoryN+ name+ '_15pm.txt'
    with open(name2, 'a') as log:
        log.write(','.join([num for num in map(str, data)])  +'\n')

def write_file_float(name, data):
    name2 = 'results' + directoryN+  name+ '_15pm.txt'
    with open(name2, 'a') as log:
        log.write(repr( data)  +'\n')

home_dir = os.getcwd()
dir_name = "/results"+directoryN
dir_name2 = home_dir + dir_name
test = os.listdir(dir_name2)
for item in test:
    if item.endswith("_15pm.txt"):
        os.remove(os.path.join(dir_name2, item))
    if item.endswith("uqr15.vtk"):
        os.remove(os.path.join(dir_name2, item))
    if item.startswith("15pm"):
        os.remove(os.path.join(dir_name2, item))

""" Parameters """

weatherInit = weather(0)
simInit = 1
simDuration = simInit # [day] init simtime
simMax =21
depth = 60
dt = 1#/24 #1h
verbose = True

# plant system 
pl = pb.MappedPlant(seednum = 2) #pb.MappedRootSystem() #pb.MappedPlant()
pl2 = pb.MappedPlant(seednum = 2) #pb.MappedRootSystem() #pb.MappedPlant()
path = "/home/rbtlm640/4UQ/CPlantBox/modelparameter/plant/"
name ="UQ_stem"# "morning_glory_UQ"#"wheat_uqr15" #"manyleaves"## "Anagallis_femina_Leitner_2010"  # Zea_mays_1_Leitner_2010

print(path + name + ".xml")

pl.readParameters(path+ name + ".xml")


#raise Exception
sdf = pb.SDF_PlantBox(np.Inf, np.Inf, depth )

pl.setGeometry(sdf) # creates soil space to stop roots from growing out of the soil


for p in pl.getOrganRandomParameter(pb.stem):
    if (p.subType > 0):
        print(p.subType, "radius", p.a, "lmax", p.lmax, p.ln, p.lb,  p.nob())

pl.initialize(verbose = True)#, stochastic = False)
pl.simulate(simDuration, True)#, "outputpm15.txt")#simDuration

Env variables at 0 d 0 hrs :
 {'TairC': 18.9, 'Qlight': 0.00048, 'cs': 0.00085, 'RH': 0.7175124027421858, 'p_mean': -339.8802949222691, 'vg': [0.059, 0.45, 0.00644, 1.503, 1]}
/home/rbtlm640/4UQ/CPlantBox/modelparameter/plant/UQ_stem.xml
1 radius 0.5 lmax 28.0 7.0 3.0Organism::readParameters: plant tag was not found in xml file, retrying with Plant 
 4.571428571428571
2 radius 0.5 lmax 21.0 3.5 2.0 6.142857142857143
OrganRandomParameter::readXML 
key seedPos.x *dparam[key] 0 
key seedPos.y *dparam[key] 0 
key seedPos.z *dparam[key] -2.5 
key firstSt *dparam[key] 0 
key firstB *dparam[key] 0 
key delayB *dparam[key] 0 
key maxB *dparam[key] 0 
key maxTi *iparam[key] 0 
key nC *dparam[key] 0 
key nz *dparam[key] 0 
key firstSB *dparam[key] 0 
key delaySB *dparam[key] 0 
key delayRC *dparam[key] 0 
key simulationTime *dparam[key] 50 
OrganRandomParameter::readXML 
key gf *iparam[key] 3 
key tropismT *iparam[key] 1 
key a *dparam[key] 0.055 
key dx *dparam[key] 1.5 
key dxMin *dparam[key] 

In [ ]:
a1 = np.array(list(map(lambda x: np.array(x), pl.segments)), dtype = np.int64)

In [2]:

pl.simulate(6, True)#, "outputpm15.txt")#simDuration

Organism::simulate: from 1 to 7 days
Root::createSegments(): length increment below dxMin threshold (7e-10 < 1) and kept in memory
stem::simulate 2 1 6 1 1.8 0 2
Stem::createSegments 
create nodes? 0 3 3 3 1 2
create nodes!  2
to create lateral 0 0 2
Stem::createLateral 2 0 0
at i 0 found 0 1
numlats 1
from stem data 4 lat 2 4 1 0 1 1 0
leaf=> lnf is 0 1
leaf=> in create lat branch 2 2 0 0 0
to std::make_shared<LeafSpecificParameter> 1 0 1 1 1
ln: 
1 
leaf created 3 1 1 1
leaf::simulate 3 1 5.33333 -0 0 0 1
to branching zone? 1 0 0 2 1
elongate before lat 0 1 0 0 1 0 1
if (length<s) {
createLateral
Leaf::createLateral 3 0 0 1
numlats 2
from leaf data 4 lat 3 4 2 0 0 1 1
to std::make_shared<LeafSpecificParameter> 2 0 1 1 0
ln: 

leaf created 4 1 0 1
leaf::simulate 4 2 5.33333 -0 0 0 1
leaf id 4 end simulate 1 0 0 0 0
2 1
from leaf data 4 lat 3 4 2 0 0 1 1
to std::make_shared<LeafSpecificParameter> 2 0 1 1 0
ln: 

leaf created 5 1 0 1
leaf::simulate 5 2 5.33333 -0 0 0 1
leaf id 5 end sim

In [3]:

np.array([org.getId() for org in pl.getOrgans()])

orgs = pl.getOrgans()
pl.getNumberOfOrgans()
#print(p.segLength())

for org in orgs:
     print("org.getId(), ",org.getId(),"ot", org.organType(),"st", org.getParameter("subType"),"length",org.getLength(),org.getLength(org.getNumberOfNodes()-1),
           "lmax",org.getParameter("lmax"),"age",org.getAge(),
          np.array( org.getOrigin()),"non", org.getNumberOfNodes(),"numKids",org.getNumberOfChildren(), "node0_id",org.getNodeId(0))
     print("     ",org.getNode(org.getNumberOfNodes() - 1))

     nns = [ np.array(org.getNode(n_))
             for n_ in range(org.getNumberOfNodes())]
     print("     ",nns)
     nns = [ org.getNodeId(n_)
             for n_ in range(org.getNumberOfNodes())]
     print("     ",nns)
     for nkid in range(org.getNumberOfChildren()):
         orgkid = org.getChild(nkid)
         print("     ","kid :",orgkid.getId(), orgkid.organType(), orgkid.getOrigin(), orgkid.getNumberOfNodes(),
               orgkid.getNumberOfChildren(),orgkid.getNodeId(0))
         print("     ",orgkid.getNode(orgkid.getNumberOfNodes() - 1))
     #print(nns)

org.getId(),  2 ot 3 st 1.0 length 12.6 12.6 lmax 28.0 age 7.0 [ 0.   0.  -2.5] non 6 numKids 4 node0_id 0
      ( 0, 0, 10.1 )
      [array([ 0. ,  0. , -2.5]), array([0. , 0. , 0.5]), array([0. , 0. , 5.5]), array([0. , 0. , 7.5]), array([0.  , 0.  , 9.85]), array([ 0. ,  0. , 10.1])]
      [0, 1, 8, 15, 22, 23]
      kid : 3 4 ( 0, 0, 0.5 ) 3 4 1
      ( -1.73895, -0.0603077, 1.48509 )
      kid : 8 4 ( 0, 0, 7.5 ) 3 4 15
      ( -1.75115, -1.0939e-17, 8.46512 )
      kid : 13 4 ( 0, 0, 9.85 ) 3 4 22
      ( -1.16024, -1.09463e-17, 10.5065 )
      kid : 18 4 ( 0, 0, 10.1 ) 1 0 23
      ( 0, 0, 10.1 )
org.getId(),  3 ot 4 st 1.0 length 2.0 2.0 lmax 2.0 age 5.333333333333334 [0.  0.  0.5] non 3 numKids 4 node0_id 1
      ( -1.73895, -0.0603077, 1.48509 )
      [array([0. , 0. , 0.5]), array([-0.85845555, -0.03016895,  1.01199991]), array([-1.73895291, -0.06030774,  1.48509192])]
      [1, 4, 7]
      kid : 4 4 ( 0, 0, 0.5 ) 2 0 1
      ( -0.783327, -1.49458e-16, 1.12161 )
      kid : 

In [6]:
ana = pb.SegmentAnalyser(pl.mappedSegments())
print(np.array([[seg.x,seg.y] for seg in pl.segments]))#.flatten())
print(np.array([[seg.x,seg.y] for seg in ana.segments]).flatten())
print(np.array([[seg.x,seg.y] for seg in ana.segments]).flatten())
ana.write("results"+directoryN+"juL"+ str(0) +".vtp", ["organType","subType"])
print(np.array([[seg.x,seg.y] for seg in ana.segments]).flatten())
print("plant()")
ana = pb.SegmentAnalyser(pl.mappedSegments())
print(np.array([[seg.x,seg.y] for seg in ana.segments]).flatten())
ana.write("results"+directoryN+"juL"+ str(1) +".vtp", ["organType","subType"])
print(np.array([[seg.x,seg.y] for seg in ana.segments]).flatten())

[[ 0  1]
 [ 1  2]
 [ 1  3]
 [ 1  4]
 [ 4  5]
 [ 4  6]
 [ 4  7]
 [ 1  8]
 [15  9]
 [15 10]
 [15 11]
 [11 12]
 [11 13]
 [11 14]
 [ 8 15]
 [22 16]
 [22 17]
 [22 18]
 [18 19]
 [18 20]
 [18 21]
 [15 22]
 [22 23]]
[ 0  1  1  2  1  3  1  4  4  5  4  6  4  7  1  8 15  9 15 10 15 11 11 12
 11 13 11 14  8 15 22 16 22 17 22 18 18 19 18 20 18 21 15 22 22 23]
[ 0  1  1  2  1  3  1  4  4  5  4  6  4  7  1  8 15  9 15 10 15 11 11 12
 11 13 11 14  8 15 22 16 22 17 22 18 18 19 18 20 18 21 15 22 22 23]
[ 0  1  1  2  1  3  1  4  4  5  4  6  4  7  1  8  9 10  9 11  9 12 12 13
 12 14 12 15  8  9 16 17 16 18 16 19 19 20 19 21 19 22  9 16 16 23]
plant()
[ 0  1  1  2  1  3  1  4  4  5  4  6  4  7  1  8 15  9 15 10 15 11 11 12
 11 13 11 14  8 15 22 16 22 17 22 18 18 19 18 20 18 21 15 22 22 23]
[ 0  1  1  2  1  3  1  4  4  5  4  6  4  7  1  8  9 10  9 11  9 12 12 13
 12 14 12 15  8  9 16 17 16 18 16 19 19 20 19 21 19 22  9 16 16 23]
construct from MappedSegments
SegmentAnalyser::SegmentAnalyser 23 24
<Points>
0

In [53]:
print(np.array([[seg.x,seg.y] for seg in pl.segments]).flatten())

[ 0  1  1  2  1  3  1  4  4  5  4  6  4  7  1  8  8  9  8 10 15 11 11 12
 11 13 11 14  8 15 15 16 15 17 22 18 18 19 18 20 18 21 15 22 22 23 22 24
 29 25 25 26 25 27 25 28 22 29]


In [54]:
print(np.array([[seg.x,seg.y] for seg in ana.segments]).flatten())

[ 0  1  1  2  1  3  1  4  4  5  4  6  4  7  1  8  8  9  8 10 11 12 12 13
 12 14 12 15  8 11 11 16 11 17 18 19 19 20 19 21 19 22 11 18 18 23 18 24
 25 26 26 27 26 28 26 29 18 25]
